## Kenvue Promotion Analysis

This analysis aims to assess promotion availability across JNJ stores, evaluating targeted promotions and measuring availability performance both at the chain level and across individual stores where the promotions apply.

Google Sheet Master - https://docs.google.com/spreadsheets/d/1ELTy7cpgwV2A57nv3URES0yHltXofTyr9XxxDUhH2KA/edit?gid=0#gid=0

### Import Packages & Library

In [1]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
# !pip install mimetypes

### Seeting up Google Sheet Credentials

In [3]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [4]:
# Python Master for ISPStore
sheet = client.open_by_key("1ELTy7cpgwV2A57nv3URES0yHltXofTyr9XxxDUhH2KA") # Master Sheets ( Dimensions Sheet)

#### Extracting google sheet tabs into dataframes

In [5]:
ISPMaster = sheet.worksheet("ISPMaster")
StoreMaster = sheet.worksheet("StoreMaster")
PromoTarget = sheet.worksheet("PromoTarget")
PositionMaster = sheet.worksheet("PositionMaster")
ExceptionCases = sheet.worksheet("ExceptionCases")


In [6]:
ISPMaster = get_as_dataframe(ISPMaster)
StoreMaster = get_as_dataframe(StoreMaster)
PromoTarget = get_as_dataframe(PromoTarget)
PositionMaster = get_as_dataframe(PositionMaster, usecols=[0,1,2])
ExceptionCases = get_as_dataframe(ExceptionCases, usecols=[0,1,2,3])

In [7]:
PositionMaster.head()

Store Code    ISP/MR  Position
0  JNJ-REL2660  JNJ-E105       1.0
1  JNJ-REL2660  JNJ-E298       2.0
2  JNJ-APL1725  JNJ-N375       1.0
3  JNJ-APL1772  JNJ-N417       1.0
4  JNJ-APL1770  JNJ-N417       1.0

### Connecting with SQL Server for Raw Promotion Data

#### SQL Connection & Data Extraction

In [8]:
server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

In [9]:
query = """
WITH S AS (
    SELECT *,
           TRY_CONVERT(date, [SubmittedDate&Time], 105) AS SubmittedDate
    FROM [PowerBI_LISV2].[JohnsonJohnsonManagement].[PromotionTrackingReportExport]
    WHERE Month = 8 
    -- and (Reason not in ('Stock not received') OR  Reason IS NULL)
      AND Year = 2025
	  and  lower(UserName) not like '%test%' and lower(LISStoreCode) NOT LIKE '%test%' and Designation != 'Supervisor'
)
SELECT *
FROM S
WHERE SubmittedDate BETWEEN '2025-08-04' AND '2025-08-21'
;

"""

In [10]:
conn = pyodbc.connect(conn_str)
df = pd.read_sql(query, conn)
# WHERE SubmittedDate BETWEEN '2025-07-06' AND '2025-07-17'
# WHERE [Month] = MONTH(GETDATE()-1) AND [Year] = YEAR(GETDATE()-1)


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_4348\1008043720.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


#### Removing the not required columns

In [11]:
# df.columns

In [12]:
df1 = df[['VisitId', 'SubmittedDate',
        'Region', 'State', 'City',
       'Username', 'FullName',
       'LISStoreCode', 'StoreName', 'Format', 'ProductCode', 'ProductName',
       'Offer', 'IsSamePromotionRunning', 'SameonPOS', 'SameonShelftalker',
       'CountOfFacing', 'Reason', 'PromotionType', 'PromotionComment',
       'PhotoLink',
       'StockCount']]

#### Filtering out the Stores which are not in the Store Master

In [13]:
StoreMaster.columns

Index(['JNJ Store Codes', 'Store Code', 'Store Name', 'Chain Name', 'Format',
       'Store Address', 'ISP/MR Code', 'ISP/MR Name', 'Contact No',
       'Designation', 'Supervisor Code', 'Supervisor', 'ZM', 'ABI Code',
       'ABI Name', 'RKAM Code', 'RKAM Name', 'Region', 'State', 'City',
       'Store Status', 'ISP/MR Status', 'Flag'],
      dtype='object')

In [14]:
StoreMaster1 = StoreMaster[['JNJ Store Codes','Supervisor','ABI Name','RKAM Name','Store Code','Chain Name','Flag']]

In [15]:
df1_StoreMaster = pd.merge(df1,StoreMaster1,left_on ='LISStoreCode',right_on='Store Code',how = 'inner')

In [16]:
# df1_StoreMaster.info() #312788 / 339612 

In [17]:
df1_StoreMaster[:10].to_csv(r'C:\Users\Administrator\LIS\Kenvue\test.csv', index=False)

### Stock Not Received Data

In [18]:
# # dfX = df[df['Reason']=='Stock not received']
# df_stocknotrecived =df[df['Reason']=='Stock not received']
# df_stocknotrecived= df_stocknotrecived[['SubmittedDate','LISStoreCode','ProductCode', 'ProductName']]

In [19]:
# df_stocknotrecived=df_stocknotrecived.drop_duplicates()

In [20]:
# (df_stocknotrecived['SubmittedDate']).count()

In [21]:
# Data = sheet.worksheet("ExceptionCases")
# # Get starting cell coordinates
# start_row = 2
# start_col = 1  # Column H is 8th

# # Clear a block if needed (optional)
# Data.batch_clear(['A2:D'])

# # Export DataFrame to Google Sheet starting at H2
# set_with_dataframe(Data, df_stocknotrecived, row=start_row, col=start_col, include_column_header=False)


In [22]:
# ExceptionCases = sheet.worksheet("ExceptionCases")
# ExceptionCases = get_as_dataframe(ExceptionCases, usecols=[0,1,2,3])

#### Filtering out the data from the Promotion Target Master

In [23]:
PromoTarget1 =  PromoTarget[['Account','PromoCode']]

In [24]:
df1_StoreMaster_Promo = pd.merge(df1_StoreMaster,PromoTarget1, how = 'inner',left_on=['ProductCode', 'Chain Name'], right_on=['PromoCode', 'Account'])

In [25]:
# df1_StoreMaster_Promo.info()

In [26]:
# df1_StoreMaster_PromoX = df1_StoreMaster_Promo[(df1_StoreMaster_Promo['Username']=='JNJ-W492') & (df1_StoreMaster_Promo['SubmittedDate']=='2025-07-08')]

In [27]:
# df1_StoreMaster_PromoX.head()

#### Date Formatting & Filtering out data which are submitted after DOL of the users

In [28]:
df1_StoreMaster_Promo['SubmittedDate'] = pd.to_datetime(df1_StoreMaster_Promo['SubmittedDate'], errors='coerce')
# Convert 'dol' to datetime, treating '-' as NaT
ISPMaster['DOL_CLEAN'] = pd.to_datetime(ISPMaster['DOL'].replace('-', pd.NaT), errors='coerce')

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_4348\2273572758.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ISPMaster['DOL_CLEAN'] = pd.to_datetime(ISPMaster['DOL'].replace('-', pd.NaT), errors='coerce')


In [29]:
# df1_StoreMaster_Promo.info()

In [30]:
df1_StoreMaster_Promo = pd.merge(df1_StoreMaster_Promo,ISPMaster[['ISP_MR_Code','DOL_CLEAN']], left_on='Username',right_on='ISP_MR_Code', how='left')

In [31]:

# Create the flag column for dol data logic
df1_StoreMaster_Promo['dol_flag'] = np.where(
    df1_StoreMaster_Promo['DOL_CLEAN'].isna(), 1,
    np.where(df1_StoreMaster_Promo['SubmittedDate'] > df1_StoreMaster_Promo['DOL_CLEAN'], 0, 1)
)

In [32]:
# df1_StoreMaster_PromoX = df1_StoreMaster_Promo[df1_StoreMaster_Promo['dol_flag']==0]

In [33]:
final_df =df1_StoreMaster_Promo[df1_StoreMaster_Promo['dol_flag']==1]

In [34]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282296 entries, 0 to 282295
Data columns (total 34 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   VisitId                 282296 non-null  int64         
 1   SubmittedDate           282296 non-null  datetime64[ns]
 2   Region                  282296 non-null  object        
 3   State                   282296 non-null  object        
 4   City                    282296 non-null  object        
 5   Username                282296 non-null  object        
 6   FullName                282296 non-null  object        
 7   LISStoreCode            282296 non-null  object        
 8   StoreName               282296 non-null  object        
 9   Format                  282296 non-null  object        
 10  ProductCode             282296 non-null  object        
 11  ProductName             282296 non-null  object        
 12  Offer                   282296

#### Data De duplication on ('SubmittedDate', 'Username', 'LISStoreCode', 'ProductName')

In [35]:
final_df = final_df.drop_duplicates(subset=['SubmittedDate', 'Username', 'LISStoreCode', 'ProductName'])

In [36]:
final_df['VisitId'].count()

282280

In [37]:
# final_df.info()

In [38]:
# 211449-21151

#### MR Positions Mapping

In [39]:
PositionMaster.head()

Store Code    ISP/MR  Position
0  JNJ-REL2660  JNJ-E105       1.0
1  JNJ-REL2660  JNJ-E298       2.0
2  JNJ-APL1725  JNJ-N375       1.0
3  JNJ-APL1772  JNJ-N417       1.0
4  JNJ-APL1770  JNJ-N417       1.0

In [40]:
final_df2 = pd.merge(final_df,PositionMaster, how = 'left', left_on = ['LISStoreCode','Username'],right_on = ['Store Code','ISP/MR'])

In [41]:
# final_df2.info()

In [42]:
final_df2['PositionRank'] = (
    final_df2.groupby(['LISStoreCode','SubmittedDate', 'ProductName'])['Position']
    .rank(method='dense')  # or 'min', 'first', 'max' based on need
)

In [43]:
final_df2['Region'] = final_df2['Region'].apply(lambda x: 'North & East' if x in ['North', 'East'] else x)
# final_df2['PhotoLink'] = final_df2['PhotoLink'].apply(lambda x: f'=HYPERLINK("{x}", "View Photo")')

In [44]:
# kunal

In [45]:
chekpoint_for_poscode = final_df2[final_df2['PositionRank'].isnull()]

In [46]:
chekpoint_for_poscode1=chekpoint_for_poscode[['Username','LISStoreCode']].drop_duplicates()

In [47]:
# Check point to gooogle sheets

Data = sheet.worksheet("PositionMaster")
# Get starting cell coordinates
start_row = 2
start_col = 8  # Column H is 8th

# Clear a block if needed (optional)
Data.batch_clear(['H2:I1000'])

# Export DataFrame to Google Sheet starting at H2
set_with_dataframe(Data, chekpoint_for_poscode1, row=start_row, col=start_col, include_column_header=True)

In [48]:
final_df2['VisitId'].count()

282280

In [49]:
final_df2 = final_df2[final_df2['PositionRank']==1]

In [50]:
final_df2 = final_df2.drop(['Store Code_x', 'Flag', 'Account','PromoCode','ISP_MR_Code','DOL_CLEAN','dol_flag','Store Code_y','ISP/MR','Position','PositionRank'], axis=1)

In [51]:
final_raw=final_df2[['VisitId','SubmittedDate','Region','State','City','JNJ Store Codes','LISStoreCode','StoreName','Chain Name','Format','Username','FullName','Supervisor','ABI Name','RKAM Name','ProductCode','ProductName','Offer','IsSamePromotionRunning','SameonPOS','SameonShelftalker','CountOfFacing','Reason','PromotionType','PromotionComment','PhotoLink','StockCount']]

In [52]:
final_raw.to_csv(r'C:\Users\Administrator\LIS\Kenvue\jnj_promoiton_final_raw.csv', index=False)

#### Aggregation on Final Raw Data & its Pivot- Unpivot

In [53]:
promotion_avail_dtlevel=final_raw.drop(['Offer','SameonPOS','SameonShelftalker','CountOfFacing','Reason','PromotionType','PromotionComment','PhotoLink','StockCount'], axis =1 )

In [54]:
promotion_avail_dtlevel.head(3)

VisitId SubmittedDate        Region      State    City JNJ Store Codes  \
0   420959    2025-08-04  North & East  Rajasthan  Jaipur            3008   
1   420959    2025-08-04  North & East  Rajasthan  Jaipur            3008   
2   420959    2025-08-04  North & East  Rajasthan  Jaipur            3008   

  LISStoreCode           StoreName Chain Name Format  Username  \
0   JNJ-DM1268  D-Mart Kalwar Road     D Mart  Hyper  JNJ-N731   
1   JNJ-DM1268  D-Mart Kalwar Road     D Mart  Hyper  JNJ-N731   
2   JNJ-DM1268  D-Mart Kalwar Road     D Mart  Hyper  JNJ-N731   

          FullName     Supervisor    ABI Name       RKAM Name ProductCode  \
0  Deepak Prajapat  Prakash yadav  Ravi Gupta  Shashank Singh   JNJPR2795   
1  Deepak Prajapat  Prakash yadav  Ravi Gupta  Shashank Singh   JNJPR2848   
2  Deepak Prajapat  Prakash yadav  Ravi Gupta  Shashank Singh   JNJPR2846   

                           ProductName IsSamePromotionRunning  
0   Stayfree D/Max All Night Ult 42Pad                    Yes  
1  Stayfree Secure Night Xxl Pads(40U)                    Yes  
2               Stayfree Secure Xl 40S                    Yes

In [55]:
# 420720
promotion_avail_dtlevelX = promotion_avail_dtlevel[promotion_avail_dtlevel['VisitId']== '420720']

In [56]:
promotion_avail_dtlevelX

Empty DataFrame
Columns: [VisitId, SubmittedDate, Region, State, City, JNJ Store Codes, LISStoreCode, StoreName, Chain Name, Format, Username, FullName, Supervisor, ABI Name, RKAM Name, ProductCode, ProductName, IsSamePromotionRunning]
Index: []

In [57]:
promotion_avail_dtlevel['ProductHeader'] = promotion_avail_dtlevel['ProductCode'] + ' (' + promotion_avail_dtlevel['ProductName'] + ')'

promotion_avail_dtlevel_pivot = promotion_avail_dtlevel.pivot_table(
    index=['VisitId','SubmittedDate','Region','State','City','JNJ Store Codes','LISStoreCode','StoreName','Chain Name','Format','Username','FullName','Supervisor','ABI Name','RKAM Name'], 
    columns='ProductHeader',
    values='IsSamePromotionRunning',
    aggfunc='first'
).reset_index()

promotion_avail_dtlevel_pivot = promotion_avail_dtlevel_pivot.fillna('-')

In [58]:
promotion_avail_dtlevel_pivot.head(3)

ProductHeader  VisitId SubmittedDate        Region           State    City  \
0               420716    2025-08-04  North & East   Uttar Pradesh   NOIDA   
1               420717    2025-08-04  North & East  Madhya Pradesh  Indore   
2               420718    2025-08-04          West     Maharashtra  Mumbai   

ProductHeader JNJ Store Codes LISStoreCode                    StoreName  \
0                        2005   JNJ-DM1264            D Mart Char Murti   
1                        4523   JNJ-DM1088  Avenuesupermartsltd.Nipaina   
2                        4038   JNJ-DM1093              Bhayander DMart   

ProductHeader Chain Name Format  ...  \
0                 D Mart  Hyper  ...   
1                 D Mart  Hyper  ...   
2                 D Mart  Hyper  ...   

ProductHeader JNJPR3121 (NeutBrightBoostResurfacingPolish75mlASO)  \
0                                                              -    
1                                                              -    
2                                                              -    

ProductHeader JNJPR3125 (Visible Repair Eye Cream 15G)  \
0                                                    -   
1                                                    -   
2                                                    -   

ProductHeader JNJPR3126 (Visible Repair Serum 30Ml)  \
0                                                 -   
1                                                 -   
2                                                 -   

ProductHeader JNJPR3127 (Visible Repair Regenerating Cream 50G)  \
0                                                             -   
1                                                             -   
2                                                             -   

ProductHeader JNJPR3131 (STAYFREE ADVANCED ULTRA SOFT XXL 42'S)  \
0                                                             -   
1                                                             -   
2                                                             -   

ProductHeader JNJPR3132 (J&J-BABY_CRM-BABY CREAM-200GM-72PK)  \
0                                                          -   
1                                                          -   
2                                                          -   

ProductHeader JNJPR3133 (J&J-BABY_SOAP-75GX3-36PK)  \
0                                                -   
1                                                -   
2                                                -   

ProductHeader JNJPR3134 (J&J-BABY_WIPES-SKINCARE-72PC-12PK)  \
0                                                         -   
1                                                         -   
2                                                         -   

ProductHeader JNJPR3135 (J AND J BABY SOAP 3+1 100GM)  \
0                                                   -   
1                                                   -   
2                                                   -   

ProductHeader JNJPR3136 (STAYFREE ADVANCED SANITARY PADS XXL 42N)  
0                                                              -   
1                                                              -   
2                                                              -   

[3 rows x 161 columns]

In [59]:
promotion_avail_dtlevel_pivot['VisitId'].count()

17058

In [60]:
id_vars = ['VisitId','SubmittedDate','Region','State','City','JNJ Store Codes','LISStoreCode','StoreName','Chain Name','Format','Username','FullName','Supervisor','ABI Name','RKAM Name']  # Add others if needed

# Melt the DataFrame
promotion_avail_dtlevel_reupivot = promotion_avail_dtlevel_pivot.melt(
    id_vars=id_vars,
    var_name='ProductHeader',
    value_name='Status'
)

In [61]:
promotion_avail_dtlevel_reupivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2490468 entries, 0 to 2490467
Data columns (total 17 columns):
 #   Column           Dtype         
---  ------           -----         
 0   VisitId          int64         
 1   SubmittedDate    datetime64[ns]
 2   Region           object        
 3   State            object        
 4   City             object        
 5   JNJ Store Codes  object        
 6   LISStoreCode     object        
 7   StoreName        object        
 8   Chain Name       object        
 9   Format           object        
 10  Username         object        
 11  FullName         object        
 12  Supervisor       object        
 13  ABI Name         object        
 14  RKAM Name        object        
 15  ProductHeader    object        
 16  Status           object        
dtypes: datetime64[ns](1), int64(1), object(15)
memory usage: 323.0+ MB


In [62]:
promotion_avail_dtlevel_reupivot['ProductCode'] = promotion_avail_dtlevel_reupivot['ProductHeader'].str.extract(r'^(.*?)\s*\(')

In [63]:
promotion_avail_dtlevel_reupivot.head(3)

VisitId SubmittedDate        Region           State    City  \
0   420716    2025-08-04  North & East   Uttar Pradesh   NOIDA   
1   420717    2025-08-04  North & East  Madhya Pradesh  Indore   
2   420718    2025-08-04          West     Maharashtra  Mumbai   

  JNJ Store Codes LISStoreCode                    StoreName Chain Name Format  \
0            2005   JNJ-DM1264            D Mart Char Murti     D Mart  Hyper   
1            4523   JNJ-DM1088  Avenuesupermartsltd.Nipaina     D Mart  Hyper   
2            4038   JNJ-DM1093              Bhayander DMart     D Mart  Hyper   

   Username                  FullName               Supervisor  \
0  JNJ-N370       Sachin Kumar Shukla       Chanky Kumar Batra   
1  JNJ-N445            Roshni Chouhan       Shailendra Raikwar   
2  JNJ-W110  Sandhya Pandurang Gavkar  Kalyani Mathurbhai Maru   

            ABI Name       RKAM Name  \
0  Vibhor Srivastava  Shashank Singh   
1       Nilesh Kumar  Shashank Singh   
2       Mudar Shamsi   Vibha Bhagwat   

                               ProductHeader Status ProductCode  
0  JNJPR2753 (Stayfree Secure Nights 18S In)      -   JNJPR2753  
1  JNJPR2753 (Stayfree Secure Nights 18S In)      -   JNJPR2753  
2  JNJPR2753 (Stayfree Secure Nights 18S In)      -   JNJPR2753

In [64]:
PromoTarget.head()

Account  PromoCode                               ProductName  \
0  Vishal Mega Mart  JNJPR2959        Stayfree-Sntry_Pd-Secure Xl 40 Pad   
1  Vishal Mega Mart  JNJPR2914      Stayfree-Sntry_Pd-Secure Night-18Pad   
2  Vishal Mega Mart  JNJPR3082  STAYFREE-SNTRY_PD-DRYM ALNGHT-28PAD-24PK   
3  Vishal Mega Mart  JNJPR2992  STAYFREE-SNTRY_PD-DRYM ALNGHT-42PAD-24PK   
4  Vishal Mega Mart  JNJPR2949   Stayfree-Sntry_Pd-Adv All Night Xxl+ 8S   

   Flag      Target_Promo  
0   1.0  =COUNTIF(A:A,A2)  
1   1.0  =COUNTIF(A:A,A3)  
2   1.0  =COUNTIF(A:A,A4)  
3   1.0  =COUNTIF(A:A,A5)  
4   1.0  =COUNTIF(A:A,A6)

In [65]:
promotion_avail_dtlevel_reupivot['key'] = promotion_avail_dtlevel_reupivot['Chain Name'] + promotion_avail_dtlevel_reupivot['ProductCode']
PromoTarget['key'] = PromoTarget['Account'] + PromoTarget['PromoCode']
valid_keys = set(PromoTarget['key'])

promotion_avail_dtlevel_reupivot['revised_status'] = promotion_avail_dtlevel_reupivot.apply(
    lambda row: 'No' if row['Status'] == '-' and row['key'] in valid_keys else row['Status'],
    axis=1
)

promotion_avail_dtlevel_reupivot.drop(columns='key', inplace=True)
PromoTarget.drop(columns='key', inplace=True)

In [66]:
promotion_avail_dtlevel_reupivot.head(2)

VisitId SubmittedDate        Region           State    City  \
0   420716    2025-08-04  North & East   Uttar Pradesh   NOIDA   
1   420717    2025-08-04  North & East  Madhya Pradesh  Indore   

  JNJ Store Codes LISStoreCode                    StoreName Chain Name Format  \
0            2005   JNJ-DM1264            D Mart Char Murti     D Mart  Hyper   
1            4523   JNJ-DM1088  Avenuesupermartsltd.Nipaina     D Mart  Hyper   

   Username             FullName          Supervisor           ABI Name  \
0  JNJ-N370  Sachin Kumar Shukla  Chanky Kumar Batra  Vibhor Srivastava   
1  JNJ-N445       Roshni Chouhan  Shailendra Raikwar       Nilesh Kumar   

        RKAM Name                              ProductHeader Status  \
0  Shashank Singh  JNJPR2753 (Stayfree Secure Nights 18S In)      -   
1  Shashank Singh  JNJPR2753 (Stayfree Secure Nights 18S In)      -   

  ProductCode revised_status  
0   JNJPR2753              -  
1   JNJPR2753              -

In [67]:
promotion_avail_dtlevel_reupivotX = promotion_avail_dtlevel_reupivot[(promotion_avail_dtlevel_reupivot['Status']=='-') & (promotion_avail_dtlevel_reupivot['revised_status']=='No')]

In [68]:
promotion_avail_dtlevel_reupivotX['revised_status'].info()

<class 'pandas.core.series.Series'>
Index: 34 entries, 340253 to 2438386
Series name: revised_status
Non-Null Count  Dtype 
--------------  ----- 
34 non-null     object
dtypes: object(1)
memory usage: 544.0+ bytes


In [69]:
promotion_avail_dtlevel_reupivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2490468 entries, 0 to 2490467
Data columns (total 19 columns):
 #   Column           Dtype         
---  ------           -----         
 0   VisitId          int64         
 1   SubmittedDate    datetime64[ns]
 2   Region           object        
 3   State            object        
 4   City             object        
 5   JNJ Store Codes  object        
 6   LISStoreCode     object        
 7   StoreName        object        
 8   Chain Name       object        
 9   Format           object        
 10  Username         object        
 11  FullName         object        
 12  Supervisor       object        
 13  ABI Name         object        
 14  RKAM Name        object        
 15  ProductHeader    object        
 16  Status           object        
 17  ProductCode      object        
 18  revised_status   object        
dtypes: datetime64[ns](1), int64(1), object(17)
memory usage: 361.0+ MB


### Identify Exception Records:

Each row in the main promotion data is compared against the exception list. </br>
A match is identified based on:</br>
1. Date (SubmittedDate)</br>
2. Store Code (LISStoreCode)</br>
3. Promotion Product Code (ProductCode)</br>
4. If a row exists in both the main data and exception list, it’s flagged with an is_exception_flag = 1. Otherwise, it’s flagged as 0.</br>

#### Filter Only Exception Records:
After flagging, only rows marked with is_exception_flag = 1 are selected for further review or export. </br>
These exception-flagged rows are exported to Google Sheets, for Validation or possible date issues in the google sheet

In [70]:
ExceptionCases['SubmittedDate'] = pd.to_datetime(ExceptionCases['SubmittedDate'].astype(str),dayfirst=True).dt.date
promotion_avail_dtlevel_reupivot['SubmittedDate'] = pd.to_datetime(promotion_avail_dtlevel_reupivot['SubmittedDate']).dt.date

# ExceptionCases['LISStoreCode'] = ExceptionCases['LISStoreCode'].str.strip().str.upper()
# promotion_avail_dtlevel_reupivot['LISStoreCode'] = promotion_avail_dtlevel_reupivot['LISStoreCode'].str.strip().str.upper()

ExceptionCases['LISStoreCode'] = ExceptionCases['LISStoreCode'].astype(str).str.strip().str.upper()
promotion_avail_dtlevel_reupivot['LISStoreCode'] = promotion_avail_dtlevel_reupivot['LISStoreCode'].astype(str).str.strip().str.upper()



promotion_avail_dtlevel_reupivot = promotion_avail_dtlevel_reupivot.merge(
    ExceptionCases,
    how='left',
    on=['SubmittedDate', 'LISStoreCode', 'ProductCode'],
    indicator='merge_flag'
)

promotion_avail_dtlevel_reupivot['is_exception_flag'] = (promotion_avail_dtlevel_reupivot['merge_flag'] == 'both').astype(int)
promotion_avail_dtlevel_reupivot.drop(columns='merge_flag', inplace=True)


In [71]:
promotion_avail_dtlevel_reupivotX =promotion_avail_dtlevel_reupivot[(promotion_avail_dtlevel_reupivot['LISStoreCode']=='JNJ-DM1147')& (promotion_avail_dtlevel_reupivot['ProductCode']=='JNJPR2846')
& (promotion_avail_dtlevel_reupivot['is_exception_flag']==1)]
promotion_avail_dtlevel_reupivotX.head(20)

Empty DataFrame
Columns: [VisitId, SubmittedDate, Region, State, City, JNJ Store Codes, LISStoreCode, StoreName, Chain Name, Format, Username, FullName, Supervisor, ABI Name, RKAM Name, ProductHeader, Status, ProductCode, revised_status, Promotion Product Name, is_exception_flag]
Index: []

[0 rows x 21 columns]

In [72]:
ExceptionCases.head()

Empty DataFrame
Columns: [SubmittedDate, LISStoreCode, ProductCode, Promotion Product Name]
Index: []

In [73]:
# Validation Code to chek the Correct ExceptionCases Mappoing

exceptions_df = promotion_avail_dtlevel_reupivot[promotion_avail_dtlevel_reupivot['is_exception_flag'] == 1]
columns_to_export = ExceptionCases.columns
export_df = exceptions_df[list(columns_to_export)]

# sheet = client.open("1ELTy7cpgwV2A57nv3URES0yHltXofTyr9XxxDUhH2KA").worksheet("Exception Cases")
Data = sheet.worksheet("ExceptionCases")
# Get starting cell coordinates
start_row = 2
start_col = 8  # Column H is 8th

# Clear a block if needed (optional)
Data.batch_clear(['H2:K1000'])

# Export DataFrame to Google Sheet starting at H2
set_with_dataframe(Data, export_df, row=start_row, col=start_col, include_column_header=True)


In [74]:
# kunal

In [75]:
# creating re pivot
df_repivot = promotion_avail_dtlevel_reupivot.drop(columns=['ProductCode', 'Status'])

# Pivot the data at 'ProductHeader'
df_pivoted_raw = df_repivot.pivot_table(
    index=['VisitId','SubmittedDate', 'Region', 'State', 'City', 'JNJ Store Codes', 'LISStoreCode', 
           'StoreName', 'Chain Name', 'Format', 'Username', 'FullName', 
           'Supervisor', 'ABI Name', 'RKAM Name'],
    columns='ProductHeader',
    values='revised_status',
    aggfunc='first'  # or use ' '.join if you expect multiple values and want to merge
)
df_pivoted_raw.reset_index(inplace=True)

In [76]:
# Creating Target Achievment Matrix ( doing all the Exception Adjutments)
filtered_df_repivot = df_repivot[(df_repivot['revised_status'] != '-') & (df_repivot['is_exception_flag'] == 0)]

agg_filtered_df_repivot = filtered_df_repivot.groupby([
    'VisitId'
]).agg(
    valid_promo_count=('revised_status', 'count'),
    #valid_promo_count = ('revised_status', lambda x: (x != '-').sum()),
    yes_promo_count   = ('revised_status', lambda x: (x == 'Yes').sum())
).reset_index()


In [77]:
df_pivoted_raw.head()

ProductHeader  VisitId SubmittedDate        Region           State     City  \
0               420716    2025-08-04  North & East   Uttar Pradesh    NOIDA   
1               420717    2025-08-04  North & East  Madhya Pradesh   Indore   
2               420718    2025-08-04          West     Maharashtra   Mumbai   
3               420719    2025-08-04          West         Gujarat  Navsari   
4               420720    2025-08-04         South       Karnataka  Raichur   

ProductHeader JNJ Store Codes LISStoreCode                     StoreName  \
0                        2005   JNJ-DM1264             D Mart Char Murti   
1                        4523   JNJ-DM1088   Avenuesupermartsltd.Nipaina   
2                        4038   JNJ-DM1093               Bhayander DMart   
3                        3928   JNJ-DM1266                 Navsari DMart   
4                        TZ03   JNJ-102271  INOK Mall Siya Talab Raichur   

ProductHeader  Chain Name Format  ...  \
0                  D Mart  Hyper  ...   
1                  D Mart  Hyper  ...   
2                  D Mart  Hyper  ...   
3                  D Mart  Hyper  ...   
4              Reliance 2  Hyper  ...   

ProductHeader JNJPR3121 (NeutBrightBoostResurfacingPolish75mlASO)  \
0                                                              -    
1                                                              -    
2                                                              -    
3                                                              -    
4                                                              -    

ProductHeader JNJPR3125 (Visible Repair Eye Cream 15G)  \
0                                                    -   
1                                                    -   
2                                                    -   
3                                                    -   
4                                                    -   

ProductHeader JNJPR3126 (Visible Repair Serum 30Ml)  \
0                                                 -   
1                                                 -   
2                                                 -   
3                                                 -   
4                                                 -   

ProductHeader JNJPR3127 (Visible Repair Regenerating Cream 50G)  \
0                                                             -   
1                                                             -   
2                                                             -   
3                                                             -   
4                                                             -   

ProductHeader JNJPR3131 (STAYFREE ADVANCED ULTRA SOFT XXL 42'S)  \
0                                                             -   
1                                                             -   
2                                                             -   
3                                                             -   
4                                                             -   

ProductHeader JNJPR3132 (J&J-BABY_CRM-BABY CREAM-200GM-72PK)  \
0                                                          -   
1                                                          -   
2                                                          -   
3                                                          -   
4                                                          -   

ProductHeader JNJPR3133 (J&J-BABY_SOAP-75GX3-36PK)  \
0                                                -   
1                                                -   
2                                                -   
3                                                -   
4                                                -   

ProductHeader JNJPR3134 (J&J-BABY_WIPES-SKINCARE-72PC-12PK)  \
0                                                         -   
1                                                         -   
2                    

In [78]:
# df_pivoted_raw.drop(columns=['Promotion Product Name','is_exception_flag'],axis=1, inplace=True)

In [79]:
# filtered_df_repivot.head()

In [80]:
agg_filtered_df_repivot2 = pd.merge(df_pivoted_raw,agg_filtered_df_repivot, how='left', on = 'VisitId')

In [81]:
agg_filtered_df_repivot2['VisitId'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 17058 entries, 0 to 17057
Series name: VisitId
Non-Null Count  Dtype
--------------  -----
17058 non-null  int64
dtypes: int64(1)
memory usage: 133.4 KB


In [82]:
agg_filtered_df_repivot2.head()

VisitId SubmittedDate        Region           State     City  \
0   420716    2025-08-04  North & East   Uttar Pradesh    NOIDA   
1   420717    2025-08-04  North & East  Madhya Pradesh   Indore   
2   420718    2025-08-04          West     Maharashtra   Mumbai   
3   420719    2025-08-04          West         Gujarat  Navsari   
4   420720    2025-08-04         South       Karnataka  Raichur   

  JNJ Store Codes LISStoreCode                     StoreName  Chain Name  \
0            2005   JNJ-DM1264             D Mart Char Murti      D Mart   
1            4523   JNJ-DM1088   Avenuesupermartsltd.Nipaina      D Mart   
2            4038   JNJ-DM1093               Bhayander DMart      D Mart   
3            3928   JNJ-DM1266                 Navsari DMart      D Mart   
4            TZ03   JNJ-102271  INOK Mall Siya Talab Raichur  Reliance 2   

  Format  ... JNJPR3126 (Visible Repair Serum 30Ml)  \
0  Hyper  ...                                     -   
1  Hyper  ...                                     -   
2  Hyper  ...                                     -   
3  Hyper  ...                                     -   
4  Hyper  ...                                     -   

  JNJPR3127 (Visible Repair Regenerating Cream 50G)  \
0                                                 -   
1                                                 -   
2                                                 -   
3                                                 -   
4                                                 -   

  JNJPR3131 (STAYFREE ADVANCED ULTRA SOFT XXL 42'S)  \
0                                                 -   
1                                                 -   
2                                                 -   
3                                                 -   
4                                                 -   

  JNJPR3132 (J&J-BABY_CRM-BABY CREAM-200GM-72PK)  \
0                                              -   
1                                              -   
2                                              -   
3                                              -   
4                                              -   

  JNJPR3133 (J&J-BABY_SOAP-75GX3-36PK)  \
0                                    -   
1                                    -   
2                                    -   
3                                    -   
4                                    -   

  JNJPR3134 (J&J-BABY_WIPES-SKINCARE-72PC-12PK)  \
0                                             -   
1                                             -   
2                                             -   
3                                             -   
4                                             -   

  JNJPR3135 (J AND J BABY SOAP 3+1 100GM)  \
0                                       -   
1                                       -   
2                                       -   
3                                       -   
4                                       -   

  JNJPR3136 (STAYFREE ADVANCED SANITARY PADS XXL 42N) valid_promo_count  \
0                                                  -                  8   
1                                                  -                  8   
2                                                  -                  8   
3                                                  -                  8   
4                                                  -                 23   

  yes_promo_count  
0               8  
1               8  
2               8  
3               8  
4              16  

[5 rows x 163 columns]

In [83]:
agg_filtered_df_repivot2.to_csv(r'C:\Users\Administrator\LIS\Kenvue\jnj_promotion_avail_dtlevel_pivot.csv', index=False)